In [ ]:
import requests      # Для запросов по API
import json          # Для обработки полученных результатов
import time          # Для задержки между запросами
import os            # Для работы с файлами
import pandas as pd  # Для формирования датафрейма с результатами


In [181]:
#pd.set_option('display.max_columns', None)

In [59]:
# получение словаря с ID стран и городов
def getAreas(area_ID):
    req = requests.get('https://api.hh.ru/areas', params={'area': area_ID})    
    data = req.content.decode()
    req.close()
    jsObj = json.loads(data)
    areas = []
    for k in jsObj:
        for i in range(len(k['areas'])):
            if len(k['areas'][i]['areas']) != 0:                      # Если у зоны есть внутренние зоны
                for j in range(len(k['areas'][i]['areas'])):
                    areas.append([k['id'], 
                                  k['name'], 
                                  k['areas'][i]['areas'][j]['id'],
                                  k['areas'][i]['areas'][j]['name']])
            else:                                                                # Если у зоны нет внутренних зон
                areas.append([k['id'], 
                              k['name'], 
                              k['areas'][i]['id'], 
                              k['areas'][i]['name']])
    return areas

In [ ]:
# получение ID нужного города
def GetCityID(city, areas):
    index = None

    for idx, item in enumerate(areas):
        found = True
        try:
            item.index(city)
        except ValueError:
            found = False

        if found != False:
            index = idx

    city_id = areas[index][2]

    return city_id

In [185]:
def query(tag, params=None):
    req = requests.get(f'https://api.hh.ru/{tag}', params=params)
    data = req.content.decode()
    req.close()
    return json.loads(data)

In [63]:
#Variables
area_ID = 113
areas = getAreas(area_ID)
city_id = GetCityID('Казань', areas)
city_id

'88'

In [187]:
# Получение словаря переменных
vars_dict = query('dictionaries')

In [188]:
vars_dict['experience']

[{'id': 'noExperience', 'name': 'Нет опыта'},
 {'id': 'between1And3', 'name': 'От 1 года до 3 лет'},
 {'id': 'between3And6', 'name': 'От 3 до 6 лет'},
 {'id': 'moreThan6', 'name': 'Более 6 лет'}]

In [202]:
# Получение вакансий
params={
        #'text': "DataScientist junior",
        'name': 'description',
        'experience': ['noExperience', 'between1And3'],
        'professional_role': '165',
        'type': 'open',
        'employment': 'full'
        }
vacancies = query('vacancies', params)

In [203]:
df = pd.DataFrame(vacancies['items'])

In [ ]:
#df[['id', 'published_at', 'employer', 'name', 'professional_roles', 'area', 'salary', 'alternate_url', 'snippet', 'experience', 'employment']]

In [ ]:
vacanciesResult['employer'] = df.employer.apply(lambda x: x['name'])
vacanciesResult['professional_roles'] = df.professional_roles.apply(lambda x: x[0]['name'])
vacanciesResult['area'] = df.area.apply(lambda x: x['name'])
vacanciesResult['snippet'] = df.snippet.apply(lambda x: x['requirement'])
vacanciesResult['experience'] = df.experience.apply(lambda x: x['name'])
vacanciesResult['employment'] = df.employment.apply(lambda x: x['name'])

In [ ]:
vacanciesResult['key_skills'] = vacanciesResult.id.apply(lambda x: query(f'vacancies/{x}')['key_skills'])

In [ ]:
vacanciesResult['key_skills'] = vacanciesResult.key_skills.apply(lambda x: [item['name'] for item in x])

In [303]:
vacanciesResult.head()

,id,published_at,employer,name,professional_roles,area,salary,alternate_url,snippet,experience,employment,key_skills
0,102074287,2024-06-17T10:24:52+0300,Jobski,Reinforcement Learning Engineer,Дата-сайентист,Москва,"{'from': 300000, 'to': 500000, 'currency': 'RU...",https://hh.ru/vacancy/102074287,Понимание принципов построения алгоритмов Alph...,Нет опыта,Полная занятость,"[RL, TensorFlow, PyTorch, Python, Java, C++, а..."
1,103649738,2024-07-10T13:21:20+0300,МТС,Стажер Data Scientist в команду [Центр маркети...,Дата-сайентист,Москва,None,https://hh.ru/vacancy/103649738,Владение Python. Знание математической статист...,Нет опыта,Полная занятость,[]
2,103603927,2024-07-09T15:57:21+0300,МООМАЙНД,Junior ML Engineer (Computer Vision),Дата-сайентист,Москва,"{'from': 60000, 'to': 100000, 'currency': 'RUR...",https://hh.ru/vacancy/103603927,Базовые знания в области машинного обучения и ...,От 1 года до 3 лет,Полная занятость,"[Python, PyTorch]"
3,101867625,2024-06-13T15:27:28+0300,ICNX,Data scientist,Дата-сайентист,Москва,None,https://hh.ru/vacancy/101867625,"Высшее образование (в области информатики, ста...",От 1 года до 3 лет,Полная занятость,"[Python, SQL, Big Data, Machine Learning, Post..."
4,103537325,2024-07-08T20:32:02+0300,Аналитический центр при Правительстве Российск...,Junior data scientist,Дата-сайентист,Москва,None,https://hh.ru/vacancy/103537325,Высшее/незаконченное высшее образование в обла...,От 1 года до 3 лет,Полная занятость,"[Python, Статистический анализ, SQL, Математич..."


In [304]:
vacanciesResult.to_excel('vacancies.xlsx', index=False)